<a href="https://colab.research.google.com/github/DARoSLab/Adv_robot_dyn_ctrl/blob/main/Lectures/MuJoCo_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**MuJoCo Example**


In [ ]:
!pip install mujoco
!pip install mujoco_mjx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 703.7/703.7 kB 37.4 MB/s eta 0:00:00


In [ ]:
#@title Import packages for plotting and creating graphics
import time
import itertools
import numpy as np
from typing import Callable, NamedTuple, Optional, Union, List

# Graphics and plotting.
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy
import mediapy as media
import matplotlib.pyplot as plt

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

# import os
# os.environ['MUJOCO_GL'] = 'egl'

print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

import mujoco
from mujoco import mjx


Installing mediapy:
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.1 MB/s eta 0:00:00
Setting environment variable to use GPU rendering:
env: MUJOCO_GL=egl


In [ ]:
!git clone https://github.com/DARoSLab/Adv_robot_dyn_ctrl

Cloning into 'Adv_robot_dyn_ctrl'...
remote: Enumerating objects: 148, done.
remote: Counting objects: 100% (148/148), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 148 (delta 53), reused 72 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (148/148), 9.79 MiB | 17.77 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [ ]:
xml_MiniArm = """
<mujoco>
  <asset>
    <mesh name="arm_base" file="/content/Adv_robot_dyn_ctrl/Meshes/RubatoArmMini/BaseLink.stl" scale="10 10 10"/>
    <mesh name="column_link" file="/content/Adv_robot_dyn_ctrl/Meshes/RubatoArmMini/ColumnLink.stl" scale="10 10 10"/>
    <mesh name="shoulder_pitch" file="/content/Adv_robot_dyn_ctrl/Meshes/RubatoArmMini/ShoulderPitch.stl" scale="10 10 10"/>
    <mesh name="Arm1" file="/content/Adv_robot_dyn_ctrl/Meshes/RubatoArmMini/ArmLink1.stl" scale="10 10 10"/>
    <mesh name="Arm2" file="/content/Adv_robot_dyn_ctrl/Meshes/RubatoArmMini/ArmLink2.stl" scale="10 10 10"/>
    <mesh name="wrist_pitch" file="/content/Adv_robot_dyn_ctrl/Meshes/RubatoArmMini/WristPitch.stl" scale="10 10 10"/>
    <mesh name="EE_link" file="/content/Adv_robot_dyn_ctrl/Meshes/RubatoArmMini/EE_Frame.stl" scale="10 10 10"/>
    <texture type="skybox" builtin="gradient" rgb1=".3 .5 .7" rgb2="0 0 0" width="32" height="512"/>
    <texture name="body" type="cube" builtin="flat" mark="cross" width="128" height="128" rgb1="0.8 0.6 0.4" rgb2="0.8 0.6 0.4" markrgb="1 1 1" random="0.01"/>
    <material name="body" texture="body" texuniform="true" rgba="0.8 0.6 .4 1"/>
    <texture name="grid" type="2d" builtin="checker" width="512" height="512" rgb1=".1 .2 .3" rgb2=".2 .3 .4"/>
    <material name="grid" texture="grid" texrepeat="1 1" texuniform="true" reflectance=".2"/>
    <material name="red_material" rgba="1 0 0 1"/>  # red
    <material name="orange_material" rgba="1 0.5 0 1"/>  # orange
    <material name="blue_material" rgba="0 0 1 1"/>  # blue
    <material name="green_material" rgba="0 1 0 1"/>  # green
    <material name="yellow_material" rgba="1 1 0 1"/>  # yellow
    <material name="purple_material" rgba="1 0 1 1"/>  # purple
    <material name="pink_material" rgba="1 0.75 0.75 1"/>  # pink
    <material name="white_material" rgba="1 1 1 1"/>  # white
  </asset>
  <visual>
    <global offwidth="700" offheight="400"/>  # Set the offscreen framebuffer height
  </visual>
  <worldbody>
    <geom name="floor" size="0 0 .05" type="plane" material="grid" condim="3"/>

    <camera name="my_camera" pos="1 0 1" euler="30 0 0"/>
    <light name="top" pos="0 0 1"/>

    <body name="base" euler="0 0 0">
      <geom type="mesh" contype="0"  conaffinity="0" group="1" density="0" mesh="arm_base" />

      <body name="colum_link" pos="-0.06 0 0">
        <joint name="base_yaw" type="hinge" axis="0 0 1" pos="0 0 0"/>
        <inertial pos="-0.031984 -1e-08 -0.0289441" quat="0.655781 0.264506 0.264514 0.65576" mass="2.428305" diaginertia="0.00176331 0.00174734 0.000400822"/>
        <geom type="mesh" contype="0"  conaffinity="0" group="1" density="0" mesh="column_link" material="orange_material"/>

        <body name="shoulder_pitch" pos="0 0 0.4">
          <joint name="shoulder_pitch" type="hinge" axis="0 1 0" pos="0 0 0"/>
          <inertial pos="-0.031984 -1e-08 -0.0289441" quat="0.655781 0.264506 0.264514 0.65576" mass="1.428305" diaginertia="0.00176331 0.00174734 0.000400822"/>
          <geom type="mesh" contype="3"  conaffinity="0" group="1" density="0" mesh="shoulder_pitch" material="blue_material"/>

          <body name="UpperArm" pos="0 0 0">
            <joint name="shoulder_roll" type="hinge" axis="1 0 0" pos="0 0 0"/>
            <inertial pos="-0.031984 -1e-08 -0.0289441" quat="0.655781 0.264506 0.264514 0.65576" mass="1.428305" diaginertia="0.00176331 0.00174734 0.000400822"/>
            <geom type="mesh" contype="3"  conaffinity="0" group="1" density="0" mesh="Arm1" material="green_material"/>

            <body name="LowerArm" pos="0.55 0 0">
              <joint name="elbow" type="hinge" axis="0 1 0" pos="0 0 0"/>
              <inertial pos="-0.031984 -1e-08 -0.0289441" quat="0.655781 0.264506 0.264514 0.65576" mass="1.428305" diaginertia="0.00176331 0.00174734 0.000400822"/>
              <geom type="mesh" contype="3"  conaffinity="0" group="1" density="0" mesh="Arm2" material="yellow_material"/>

              <body name="WristPitch" pos="0.45 0 0">
                <joint name="wrist_pitch" type="hinge" axis="0 1 0" pos="0 0 0"/>
                <inertial pos="-0.031984 -1e-08 -0.0289441" quat="0.655781 0.264506 0.264514 0.65576" mass="0.8" diaginertia="0.00176331 0.00174734 0.000400822"/>
                <geom type="mesh" contype="3"  conaffinity="0" group="1" density="0" mesh="wrist_pitch" material="purple_material"/>

                <body name="EE_Frame" pos="0.09 0 0">
                  <joint name="wrist_roll" type="hinge" axis="1 0 0" pos="0 0 0"/>
                  <inertial pos="0.03 0 0" quat="0.655781 0.264506 0.264514 0.65576" mass="0.3" diaginertia="0.00176331 0.00174734 0.000400822"/>
                  <geom type="mesh" contype="3"  conaffinity="0" group="1" density="0" mesh="EE_link" material="pink_material"/>
                </body>
              </body>
            </body>
          </body>
        </body>
      </body>
    </body>
  </worldbody>
<!-- Add actuators for the joints -->
  <actuator>
    <motor joint="base_yaw" ctrlrange="-50 50" gear="1"/>
    <motor joint="shoulder_pitch" ctrlrange="-50 50" gear="1"/>
    <motor joint="shoulder_roll" ctrlrange="-50 50" gear="1"/>
    <motor joint="elbow" ctrlrange="-50 50" gear="1"/>
    <motor joint="wrist_pitch" ctrlrange="-20 20" gear="1"/>
    <motor joint="wrist_roll" ctrlrange="-20 20" gear="1"/>
  </actuator>
</mujoco>
"""

# Make model, data, and renderer
mj_model = mujoco.MjModel.from_xml_string(xml_MiniArm)
mj_data = mujoco.MjData(mj_model)
renderer = mujoco.Renderer(mj_model, width=700, height=400)

In [ ]:
mjx_model = mjx.put_model(mj_model)
mjx_data = mjx.put_data(mj_model, mj_data)
print(mj_data.qpos, type(mj_data.qpos))
print(mjx_data.qpos, type(mjx_data.qpos), mjx_data.qpos.devices())

# enable joint visualization option:
scene_option = mujoco.MjvOption()
scene_option.flags[mujoco.mjtVisFlag.mjVIS_JOINT] = False

duration = 4.0  # (seconds)
framerate = 40  # (Hz)

frames = []
mujoco.mj_resetData(mj_model, mj_data)

# Define PD controller gains
kp = 250  # Proportional gain
kd = 3.5  # Derivative gain
desired_position = np.array([0.0, -1.0, 0.0, 1.5, -1.0, 0.0])  # Example desired position

while mj_data.time < duration:# Calculate joint position error
  desired_position[1] = -1.0 + 0.5 * np.sin(mj_data.time * 2 * np.pi / duration)
  position_error = desired_position - mj_data.qpos

  # Calculate joint velocity
  velocity = mj_data.qvel

  # Calculate control signal
  control_signal = kp * position_error - kd * velocity

  # Apply control signal to joints
  mj_data.ctrl = control_signal


  mujoco.mj_step(mj_model, mj_data)
  if len(frames) < mj_data.time * framerate:
    # mj_data.cam_pos = np.array([1, 0, 1])  # Set camera position
    # mj_data.cam_quat = np.array([1, 0, 0, 0])  # Set camera orientation
    renderer.update_scene(mj_data, scene_option=scene_option)
    # renderer.render(camera_id=0, camera_pos=[1, 0, 1], camera_quat=[1, 0, 0, 0])
    pixels = renderer.render()
    frames.append(pixels)

# Simulate and display video.
media.show_video(frames, fps=framerate)

/usr/local/lib/python3.10/dist-packages/mujoco/mjx/_src/mesh.py:177: UserWarning: Mesh "column_link" has a coplanar face with more than 20 vertices. This may lead to performance issues and inaccuracies in collision detection. Consider decimating the mesh.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/mujoco/mjx/_src/mesh.py:177: UserWarning: Mesh "shoulder_pitch" has a coplanar face with more than 20 vertices. This may lead to performance issues and inaccuracies in collision detection. Consider decimating the mesh.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/mujoco/mjx/_src/mesh.py:177: UserWarning: Mesh "Arm1" has a coplanar face with more than 20 vertices. This may lead to performance issues and inaccuracies in collision detection. Consider decimating the mesh.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/mujoco/mjx/_src/mesh.py:177: UserWarning: Mesh "Arm2" has a coplanar face with more than 20 vertices. This may lead to performance issues and ina

[0. 0. 0. 0. 0. 0.] <class 'numpy.ndarray'>
[0. 0. 0. 0. 0. 0.] <class 'jaxlib.xla_extension.ArrayImpl'> {CudaDevice(id=0)}


This browser does not support the video tag.
